# Initialize Environment

In [ ]:
import tensorflow as tf
import skopt
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import random
import os
from shutil import rmtree
import pickle

In [ ]:
%matplotlib inline
%load_ext tensorboard.notebook

In [ ]:
LOGS_PATH = os.path.join('.', 'logs', 'fit', 'petimages')
MODEL_FILE_NAME = 'keras_petimages_cnn.model'
DATA_X_PATH = 'petimages.x.pickle'
DATA_Y_PATH = 'petimages.y.pickle'

# Load Data

In [ ]:
def load_data(path):
    pickle_in = open(path, 'rb')
    result = pickle.load(pickle_in)
    pickle_in.close()
    return result

In [ ]:
if not os.path.isfile(DATA_X_PATH):
    raise f'Data file "{DATA_Y_PATH}" does not exist' \
        + 'download petimage dataset from kaggle and' \
        + 'use tools/load_image_data.py to perpare the data'
if not os.path.isfile(DATA_Y_PATH):
    raise f'Data file "{DATA_Y_PATH}" does not exist' \
        + 'download petimage dataset from kaggle and' \
        + 'use tools/load_image_data.py to perpare the data'
x = load_data(DATA_X_PATH)
y = load_data(DATA_Y_PATH)

In [ ]:
print(x.shape[1:])

In [ ]:
sample_pos = random.randint(1, x.shape[0])
print(f'Image at {sample_pos}')
sample_image = x[sample_pos - 1]
if len(sample_image.shape) == 3 and sample_image.shape[2] == 1:
    sample_image = sample_image.reshape(sample_image.shape[:2])
plt.imshow(sample_image, cmap = plt.cm.binary)

# Normalize Data
Data is already normalized

# Define Model

In [ ]:
def create_model(learning_rate, num_conv_layers, exp_num_filters, kernel_radius, num_dense_layers, num_dense_nodes, activation):
    kernel_shape = tuple([kernel_radius * 2 - 1] * 2)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(2**exp_num_filters, kernel_shape, input_shape=x.shape[1:]))
    model.add(tf.keras.layers.Activation(activation))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    for l in range(num_conv_layers - 1):
        model.add(tf.keras.layers.Conv2D(2**(exp_num_filters + l), kernel_shape))
        model.add(tf.keras.layers.Activation(activation))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    for i in range(num_dense_layers - 1):
        model.add(tf.keras.layers.Dense(num_dense_nodes, activation=activation))
    if num_dense_layers > 1:
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

# Train & Verify Model

In [ ]:
def log_dir_path(learning_rate, num_conv_layers, exp_num_filters, kernel_radius, num_dense_layers, num_dense_nodes, activation):
    time_stamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    log_dir_name_pattern = "lr{0:.0e}_cl{1}_e{2}_k{3}x{3}_dl{4}_n{5}_a{6}_{7}"
    kernel_width = kernel_radius * 2 - 1
    log_dir_name = log_dir_name_pattern.format(learning_rate,
                                               num_conv_layers,
                                               exp_num_filters,
                                               kernel_width,
                                               num_dense_layers,
                                               num_dense_nodes,
                                               activation,
                                               time_stamp)
    log_dir = os.path.join(LOGS_PATH, log_dir_name)

    return log_dir
    

In [ ]:
def fit_model(learning_rate, num_conv_layers, exp_num_filters, kernel_radius, num_dense_layers, num_dense_nodes, activation):
    model = create_model(learning_rate=learning_rate,
                         num_conv_layers=num_conv_layers,
                         exp_num_filters=exp_num_filters,
                         kernel_radius=kernel_radius, 
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation)
    
    log_dir = log_dir_path(learning_rate,
                           num_conv_layers,
                           exp_num_filters,
                           kernel_radius, 
                           num_dense_layers,
                           num_dense_nodes,
                           activation)
    
    tensor_board_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                           histogram_freq=1)
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                               patience=2,
                                                               restore_best_weights=True)
   
    history = model.fit(x=x,
                        y=y,
                        validation_split=0.3,
                        epochs=20, 
                        callbacks=[tensor_board_callback,
                                   early_stopping_callback])
    accuracy = history.history['val_accuracy'][early_stopping_callback.stopped_epoch]
    
    return model, accuracy

In [ ]:
# Clear any logs from previous runs
if os.path.isdir(LOGS_PATH):
    rmtree(LOGS_PATH, ignore_errors=True)

In [ ]:
initial_parameters = [1e-3, 2, 5, 3, 2, 128, 'relu']
model, accuracy = fit_model(*initial_parameters)
print()
print("Accuracy: {0:.2%}".format(accuracy))
del model
tf.keras.backend.clear_session()

# Hyperparameter Optimiziation
Using Bayesian Optimiziation

In [ ]:
dim_learning_rate = skopt.space.Real(low=1e-6, high=1e-1, prior='log-uniform', name='learning_rate')
dim_num_conv_layers = skopt.space.Integer(low=1, high=3, name='num_conv_layers')
dim_exp_num_filters = skopt.space.Integer(low=3, high=7, name='exp_num_filters')
dim_kernel_radius = skopt.space.Integer(low=2, high=4, name='kernel_radius')
dim_num_dense_layers = skopt.space.Integer(low=0, high=3, name='num_dense_layers')
dim_num_dense_nodes = skopt.space.Integer(low=5, high=512, name='num_dense_nodes')
dim_activation = skopt.space.Categorical(categories=['relu', 'sigmoid'], name='activation')
dimensions = [
    dim_learning_rate,
    dim_num_conv_layers,
    dim_exp_num_filters,
    dim_kernel_radius,
    dim_num_dense_layers,
    dim_num_dense_nodes,
    dim_activation]

In [ ]:
runs = {
    'accuracy': [],
    'learning_rate': [],
    'num_conv_layers': [],
    'exp_num_filters': [],
    'num_dense_layers': [],
    'num_dense_nodes': [],
    'activation': []
}
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_conv_layers, exp_num_filters, kernel_radius, num_dense_layers, num_dense_nodes, activation):
    
    print(f'run {len(runs["accuracy"]) + 1}')
    print()
    print('Learning rate: {0:.1e}'.format(learning_rate))
    print('Number of convolution layers:', num_conv_layers)
    print('Exponent of size of first convolution layer:', exp_num_filters)
    print('Kernel radius:', kernel_radius)
    print('Number of dense layers:', num_dense_layers)
    print('Number of dense nodes:', num_dense_nodes)
    print('Activation function:', activation)
    print()
    
    model, accuracy = fit_model(learning_rate=learning_rate,
                                num_conv_layers=num_conv_layers,
                                exp_num_filters=exp_num_filters,
                                kernel_radius=kernel_radius,
                                num_dense_layers=num_dense_layers,
                                num_dense_nodes=num_dense_nodes,
                                activation=activation)
    
    global best_run
    
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    
    best_accuracy = max(runs['accuracy'] or [0.0])
    if accuracy > best_accuracy:
        print("Best Accuracy so far!".format(best_accuracy))
        model.save(MODEL_FILE_NAME)
    else:
        print("Best Accuracy so far: {0:.2%}".format(best_accuracy))

    print()

    runs['accuracy'].append(accuracy)
    runs['learning_rate'].append(learning_rate)
    runs['num_conv_layers'].append(num_conv_layers)
    runs['exp_num_filters'].append(exp_num_filters)
    runs['num_dense_layers'].append(num_dense_layers)
    runs['num_dense_nodes'].append(num_dense_nodes)
    runs['activation'].append(activation)

    del model
    tf.keras.backend.clear_session()
    
    # Scikit-optimize tries to find a set of hyper-parameters with the LOWEST fitness-value
    return -accuracy

In [ ]:
search_result = skopt.gp_minimize(func=fitness,
                                  dimensions=dimensions,
                                  acq_func='EI',
                                  x0=initial_parameters)

In [ ]:
df = pd.DataFrame(runs)
df.sort_values('accuracy', ascending=False)

In [ ]:
%tensorboard --logdir './logs/fit/petimages'

# Use Model

In [ ]:
!test_petimages